# Parsing basics: JS and TS



In [23]:
from pathlib import Path
from tree_sitter import Language, Parser
from traverse_treesitter_tree import traverse_treesitter_tree
import json

In [2]:
# build treesitter parser
Language.build_library(
    'build/langs.so',
    [
        'extern/tree-sitter-javascript'
        # 'extern/tree-sitter-typescript'
    ]
)

False

In [3]:
# load TS and JS parsers
JS_LANG = Language('build/langs.so', 'javascript')
# TS_LANG = Language('build/langs.so', 'typescript')
js_parser = Parser()
js_parser.set_language(JS_LANG)
# ts_parser = Parser().set_language(TS_LANG)

In [4]:
# get javascript files from example/js, excluding from node_modules and hidden files (.)
js_files = []
for file in Path('example/js').glob('**/*.js'):
    if not file.name.startswith('.') and 'node_modules' not in str(file):
        js_files.append(file)

In [5]:
# display first 5 files newline separated
print('\n'.join(str(file) for file in js_files[:5]))

example/js/CodeRibbon/lib/code-ribbon-tips-view.js
example/js/CodeRibbon/lib/cr-common.js
example/js/CodeRibbon/lib/cr-base.js
example/js/CodeRibbon/lib/code-ribbon-statusbar-element.js
example/js/CodeRibbon/lib/code-ribbon-ribbon-container.js


In [6]:
# parse each file
parsed_js_files = []
for file in js_files:
    try:
        with open(file, 'r') as f:
            parsed_js_files.append(js_parser.parse(bytes(f.read(), 'utf-8')))
    except Exception as e:
        print(f'Error parsing {file}: {e}')

In [7]:
for node in traverse_treesitter_tree(parsed_js_files[-3]):
    print(node.text, node.type)

b"'use babel';\n\nimport CodeRibbon from '../lib/code-ribbon';\n\n// Use the command `window:run-package-specs` (cmd-alt-ctrl-p) to run specs.\n//\n// To run a specific `it` or `describe` block add an `f` to the front (e.g. `fit`\n// or `fdescribe`). Remove the `f` to unfocus the block.\n\ndescribe('CodeRibbon', () => {\n  let workspaceElement, activationPromise;\n\n  beforeEach(() => {\n    workspaceElement = atom.views.getView(atom.workspace);\n    activationPromise = atom.packages.activatePackage('code-ribbon');\n  });\n\n  describe('when the code-ribbon:toggle event is triggered', () => {\n    it('hides and shows the modal panel', () => {\n      // Before the activation event the view is not on the DOM, and no panel\n      // has been created\n      expect(workspaceElement.querySelector('.code-ribbon')).not.toExist();\n\n      // This is an activation event, triggering it will cause the package to be\n      // activated.\n      atom.commands.dispatch(workspaceElement, 'code-ribbon:

In [8]:
# get require statements from each file
require_statements = []
for tree in parsed_js_files:
    for node in traverse_treesitter_tree(tree):
        if node.type == 'call_expression' and 'require' in str(node.text):
            require_statements.append(node.parent)

In [9]:
# print first 5 require statements
print('\n'.join(str(node.text) for node in require_statements[:5]))

b"{\n  backgroundTipsView\n} = require('./cr-base')"
b"{\n  crdebug,\n  crlogger,\n  fuzzyFinderProjectView,\n  get_cr_panecontainer,\n  get_cr_module\n} = require('./cr-base')"
b"scrollIntoView = require('scroll-into-view')"
b'backgroundTipsView = require(backgroundTipsPackage.path + "/lib/background-tips-view.js")'
b'fuzzyFinderProjectView = require("./fuzzyfinder/cr-project-view")'


In [10]:
# get import statements from each file
import_statements = []
for tree in parsed_js_files:
    for node in traverse_treesitter_tree(tree):
        if node.type == 'import_statement':
            import_statements.append(node)

In [11]:
# print first 5 import statements
print('\n'.join(str(node.text) for node in import_statements[:5]))

b"import CodeRibbon from '../lib/code-ribbon';"
b"import CodeRibbonView from '../lib/code-ribbon-view';"


In [12]:
# use traverse_treesitter_tree to get all nodes that call a method
calls = []
for tree in parsed_js_files:
    for node in traverse_treesitter_tree(tree):
        if node.type == 'call_expression':
            calls.append(node)

In [13]:
len(calls)

1261

In [14]:
# print the first 5 calls
for call in calls[:5]:
    print(call.text)

b"require('./cr-base')"
b'super()'
b'Math.round(Math.random() * len)'
b'Math.random()'
b"this.message.classList.remove('fade-in')"


In [19]:
# get all package.json files
package_json_files = []
for file in Path('example/js').glob('**/package.json'):
    if not file.name.startswith('.') and 'node_modules' not in str(file):
        package_json_files.append(file)

In [21]:
# print first 5 package.json files
print('\n'.join(str(file) for file in package_json_files[:5]))

example/js/CodeRibbon/package.json


In [24]:
# read each package.json file and get dependencies
dependencies = []
for file in package_json_files:
    try:
        with open(file, 'r') as f:
            dependencies.append(json.load(f)['dependencies'])
    except Exception as e:
        print(f'Error parsing {file}: {e}')

In [25]:
dependencies

[{'async': '0.2.6',
  'atom-select-list': '^0.7.0',
  'etch': '^0.14.1',
  'fs-plus': '^3.0.0',
  'fuzzaldrin': '^2.0',
  'fuzzaldrin-plus': '^0.6.0',
  'humanize-plus': '~1.8.2',
  'minimatch': '~3.0.3',
  'object-hash': '^1.3.1',
  'scroll-into-view': '^1.16.0',
  'temp': '^0.8.4',
  'underscore-plus': '^1.7.0',
  'vscode-ripgrep': '^1.13.2',
  'wrench': '^1.5'}]